<a href="https://colab.research.google.com/github/MocT117/Another-one-/blob/master/MADFINDER1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import re
from datetime import datetime

# 1. Configuración general
INPUT_FOLDER = 'raw_data/'
OUTPUT_FOLDER = 'output/'

# 2. Extracción de fechas MAD y guardar historial
def extraer_fechas_mad(texto):
    """
    Extrae TODAS las fechas tipo MAD o NEW MAD en ORDEN de aparición.
    Devuelve fechas en formato YYYY-MM-DD.
    """
    if not isinstance(texto, str):
        return []

    patron = r'(?:MAD|NEW MAD)[:\s]*([0-9]{1,2}[./][0-9]{1,2}(?:[./][0-9]{2,4})?)'
    matches = re.findall(patron, texto, flags=re.IGNORECASE)

    fechas_limpias = []
    for fecha in matches:
        try:
            fecha = fecha.replace('/', '.')
            partes = fecha.split('.')
            if len(partes) == 2:
                dia, mes = partes
                año = '2025'
            elif len(partes) == 3:
                dia, mes, año = partes
                if len(año) == 2:
                    año = '20' + año
            else:
                continue
            fecha_formateada = datetime(int(año), int(mes), int(dia)).strftime('%Y-%m-%d')
            fechas_limpias.append(fecha_formateada)
        except Exception as e:
            print(f"⚠️ Error al convertir '{fecha}' → {e}")
            continue

    return fechas_limpias

# 3. Función para procesar cada fila con historial
def obtener_info_mad(row):
    """
    Devuelve:
      - La PRIMERA MAD encontrada (vigente)
      - El total de MADs en el texto (contador)
      - El historial de todas las MADs encontradas (como string separado por '|')
    Si no hay MAD, toma 'Material staging' y cambios = 0.
    """
    fechas = extraer_fechas_mad(row['Text sales order'])
    if fechas:
        mad_vigente = fechas[0]     # PRIMERA aparición
        cambios = len(fechas)       # Total de MADs
        historial = '|'.join(fechas)
        return mad_vigente, cambios, historial

    # Si no hay MAD en el texto, usa Material staging
    fecha_staging = row.get('Material staging')
    try:
        if isinstance(fecha_staging, pd.Timestamp):
            return fecha_staging.strftime('%Y-%m-%d'), 0, ''
        elif isinstance(fecha_staging, str) and fecha_staging.strip() != '':
            fecha = pd.to_datetime(fecha_staging, errors='coerce')
            if pd.notnull(fecha):
                return fecha.strftime('%Y-%m-%d'), 0, ''
    except Exception as e:
        print(f"Error con fecha staging: {e}")
    return None, 0, ''

# 4. Crea la key compuesta
def crear_columna_key(df):
    df = df.copy()
    df['SalesOrder_clean'] = df['Sales order no.'].astype(str).str.lstrip('0')
    df['MaterialNumber_clean'] = df['Material number'].astype(str).str.lstrip('0')
    df['Key'] = df['SalesOrder_clean'] + '_' + df['MaterialNumber_clean']
    return df

# 5. Flujo principal
def main():
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)

    archivos = [f for f in os.listdir(INPUT_FOLDER) if f.endswith('.xlsx')]
    df_total = pd.DataFrame()
    for archivo in archivos:
        ruta = os.path.join(INPUT_FOLDER, archivo)
        try:
            df = pd.read_excel(ruta, engine='openpyxl')
            df_total = pd.concat([df_total, df], ignore_index=True)
        except Exception as e:
            print(f"Error al leer {archivo}: {e}")

    columnas_esperadas = ['Sales order no.', 'Material number', 'Text sales order', 'Material staging']
    for col in columnas_esperadas:
        if col not in df_total.columns:
            raise ValueError(f"Falta la columna requerida: {col}")

    df_total = crear_columna_key(df_total)

    # Aplicar la función con historial
    df_total[['Ultima_Fecha_MAD', 'Cambios_MAD', 'Historial_MADs']] = df_total.apply(
        lambda row: pd.Series(obtener_info_mad(row)),
        axis=1
    )

    # Guardar resultado
    nombre_archivo = f'consolidado_{datetime.today().strftime("%Y%m%d")}.xlsx'
    ruta_salida = os.path.join(OUTPUT_FOLDER, nombre_archivo)
    df_total.to_excel(ruta_salida, index=False)
    print(f"✅ Archivo exportado: {ruta_salida}")

if __name__ == '__main__':
    main()
